# BME-Crete Master Thesis
### by **Stylianos Zafeiris**

#### Imports and necessary variable declarations

In [1]:
from src.main import *

#### Read original PICAI dataset

In [3]:
dataService = PicaiDataService()

# Extract radiomics from images or read them
if os.path.exists(conf.PICAI_RADIOMICS_FILE):
    radiomicFeatures = dataService.readRadiomics(conf.PICAI_RADIOMICS_FILE)
else:
    radiomicFeatures = dataService.extractRadiomics(conf.PICAI_NIFTI_IMAGES_DIR, conf.PICAI_RADIOMICS_FILE)

# Create labels for stratification
picaiMetadata = dataService.getMetadata(conf.PICAI_METADATA_PATH)    
jointDfs = pd.merge(picaiMetadata, radiomicFeatures, on='Patient_Id')
conditions = [
    (jointDfs['Label'] == 2) & (jointDfs['MagneticFieldStrength'] == 1.5),
    (jointDfs['Label'] == 2) & (jointDfs['MagneticFieldStrength'] == 3),
    (jointDfs['Label'] > 2)  & (jointDfs['MagneticFieldStrength'] == 1.5),
    (jointDfs['Label'] > 2)  & (jointDfs['MagneticFieldStrength'] == 3),
]
jointDfs['StratifiedLabels'] = np.select(conditions, [0, 1, 2, 3])
yStrat = jointDfs['StratifiedLabels'].to_numpy()
# log.debug(np.unique(yStrat, return_counts=True))

# Get features and original labels
patientIds = radiomicFeatures.pop('Patient_Id').to_list()
labels = radiomicFeatures.pop('Label')
radiomicFeaturesNames = radiomicFeatures.columns.to_list()
X = radiomicFeatures.to_numpy()
y = np.copy(labels)
y[y == 2] = 0   # 0: ISUP = 2,
y[y > 2] = 1    # 1: ISUP > 2

# Configure evaluation
args = {
    'patientIds': patientIds,
    'radiomicFeaturesNames': radiomicFeaturesNames,
    'featureStart': 5,
    'featureStep': 10,
    'featureStop': 1132,
}
evaluator = GridSearchNestedCVEvaluation(**args)
send_to_telegram("Evaluation started.")
evaluator.evaluate(X, y, yStrat)
send_to_telegram("Evaluation ended.")

Read csv radiomic file


AttributeError: 'GridSearchNestedCVEvaluation' object has no attribute 'evaluate'